In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from keras.models import Sequential
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense
from keras import layers
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
# creating image size parameter
img_size = 48

# creating batch size variable
batch = 64

In [3]:
# setting earlystopping
early = EarlyStopping(monitor = 'val_loss',
              mode = 'min',
              patience = 3, 
              restore_best_weights = True)

# setting modelcheckpoint
check = ModelCheckpoint('./m_checkpoint',
               monitor = 'val_loss',
               save_best_only = True,
               mode = 'min')

# creating list for callbacks
call_list = [early, check]

In [4]:
# loading data from directory
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range = 20,
        validation_split = .3,
        horizontal_flip=True)

train_generator = datagen.flow_from_directory(
        '/kaggle/input/granite_slabs/granite_slabs',
        subset = 'training',
        target_size=(img_size, img_size),
        batch_size=batch,
        color_mode = 'rgb',
        class_mode= 'categorical')

validation_generator = datagen.flow_from_directory(
        '/kaggle/input/granite_slabs/granite_slabs',
        subset = 'validation',
        target_size=(img_size, img_size),
        batch_size=batch,
        class_mode= 'categorical')

Found 452 images belonging to 6 classes.
Found 191 images belonging to 6 classes.


In [5]:
# Instantiating model
model = Sequential()

# input Convolutional layer
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))



# # Adding 1st conv layer
model.add(Conv2D(64,
                kernel_size = 3,
                activation = 'relu',
                kernel_regularizer = regularizers.l2(.01)))

# adding pooling
model.add(MaxPooling2D(pool_size = 3))

# adding dropout
model.add(Dropout(rate = .5))

# Adding 2nd conv layer
model.add(Conv2D(128,
                 kernel_size = 3,
                 activation = 'relu',
                 kernel_regularizer = regularizers.l2(.01)))

# # Adding 2nd max pooling
# model.add(MaxPooling2D(pool_size= 2))

# # adding dropout
# model.add(Dropout(rate = .25))
          
# # Adding 3rd conv layer
# model.add(Conv2D(256,
#                  kernel_size=3,
#                  activation='relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 3rd max pooling
# model.add(MaxPooling2D(pool_size= 2))

# # Adding 4th conv layer
# model.add(Conv2D(256,
#                  kernel_size=3,
#                  activation='relu', 
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 4th max pooling
# model.add(MaxPooling2D(pool_size= 2))

# # Adding 5th conv layer
# model.add(Conv2D(256,
#                  kernel_size=1,
#                  activation='relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 5th max pooling
# model.add(MaxPooling2D(pool_size= 2))

# # Adding 6th conv layer
# model.add(Conv2D(256,
#                  kernel_size=1,
#                  activation='relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# Adding 6th max pooling
model.add(MaxPooling2D(pool_size= 4))

# adding dropout
model.add(Dropout(rate = .25))

# # Adding 7th conv layer
# model.add(Conv2D(256,
#                  kernel_size=1,
#                  activation='relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 7th max pooling
# model.add(MaxPooling2D(pool_size= 2))

model.add(layers.Flatten())

# model.add(Dense(256,
#                activation = 'relu'))
# model.add(Dense(128,
#                activation = 'relu'))
# model.add(Dense(64,
#                activation = 'relu'))
model.add(Dense(128,
               activation = 'relu',
               kernel_regularizer = regularizers.l2(.01)))
model.add(layers.Dense(6, activation = 'softmax',
                      kernel_regularizer = regularizers.l2(.01)))

In [6]:
# compiling model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],)

In [7]:
# checking summary of model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 3, 128)         0         
__________

In [8]:
# fitting model
history = model.fit_generator(
        train_generator,
        steps_per_epoch=50,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=12,
        workers = -1,
        verbose = 1,
        callbacks = call_list)

Epoch 1/12
50/50 [==============================] - 790s 16s/step - loss: 3.8511 - acc: 0.1904 - val_loss: 2.5979 - val_acc: 0.3194
Epoch 2/12
50/50 [==============================] - 779s 16s/step - loss: 2.1395 - acc: 0.3047 - val_loss: 1.9038 - val_acc: 0.4005
Epoch 3/12
50/50 [==============================] - 787s 16s/step - loss: 1.6923 - acc: 0.4014 - val_loss: 1.8495 - val_acc: 0.4437
Epoch 4/12
50/50 [==============================] - 780s 16s/step - loss: 1.4911 - acc: 0.4939 - val_loss: 1.6447 - val_acc: 0.4398
Epoch 5/12
50/50 [==============================] - 796s 16s/step - loss: 1.4299 - acc: 0.5356 - val_loss: 1.6762 - val_acc: 0.4542
Epoch 6/12
50/50 [==============================] - 794s 16s/step - loss: 1.3885 - acc: 0.5152 - val_loss: 1.7202 - val_acc: 0.3508
Epoch 7/12
50/50 [==============================] - 786s 16s/step - loss: 1.3794 - acc: 0.5361 - val_loss: 1.5206 - val_acc: 0.4516
Epoch 8/12
50/50 [==============================] - 791s 16s/step - loss: 1.

In [9]:
feat_extractor = Model(model.input, outputs=model.get_layer("dense_2").output)

In [10]:
feat_extractor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
__________

In [11]:
# saving model
# model.save('./model')

In [12]:
# evaluating model performance
model.evaluate_generator(train_generator, steps = 5, workers = -1, verbose = 1)

5/5 [==============================] - 65s 13s/step


[1.287302693953881, 0.6076923076923076]